In [ ]:
from datetime import datetime, timedelta

from metpy.plots import declarative
from metpy.units import units
from siphon.catalog import TDSCatalog
import xarray as xr

In [ ]:
# Set the date/time of the model run
### SET THE APPROPRIATE DATE/TIME ###
date = datetime(2019, 3, 14, 0)
#####################################

# Remote access to the dataset from the RDA site
catUrl = "https://rda.ucar.edu/thredds/catalog/aggregations/g/ds083.2/2/catalog.xml"
datasetName = "Full Collection Dataset"
catalog = TDSCatalog(catUrl)
dataset = catalog.datasets[datasetName]
ds = dataset.remote_access(use_xarray=True)

# Subset data to be just over the U.S. for plotting purposes
ds = ds.sel(lat=slice(70,10), lon=slice(360-150, 360-55))

In [ ]:
# Set the plot time with forecast hours
plot_time = date + timedelta(hours=0)

### SET THE APPROPRIATE LEVEL ###
plot_level = 300 * units.hPa
#################################

# Set attributes for plotting contours
cntr = declarative.ContourPlot()
cntr.data = ds
cntr.field = 'Geopotential_height_isobaric'
cntr.level = plot_level
cntr.time = plot_time

### SET THE CONTOUR INTERVAL ###
cntr.contours = list(range(0, 10000, 120))
################################

cntr.clabels = True

# Set attributes for plotting Temperature Contours
cfill = declarative.FilledContourPlot()
cfill.data = ds
cfill.field = 'Temperature_isobaric'
cfill.level = plot_level
cfill.time = plot_time

### SET THE CONTOUR INTERVAL ###
cfill.contours = list(range(-46, 47, 2))
################################

cfill.colormap = 'coolwarm'
cfill.colorbar = 'horizontal'
cfill.plot_units = 'degC'

# Add wind barbds
barbs = declarative.BarbPlot()
barbs.data = ds
barbs.time = plot_time
barbs.field = ['u-component_of_wind_isobaric', 'v-component_of_wind_isobaric']
barbs.level = plot_level
barbs.skip = (4, 4)
barbs.plot_units = 'knot'

# Set the attributes for the map
# and put the contours on the map
panel = declarative.MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']

#### ADD A TITLE ###
panel.title = f''
####################

panel.plots = [cfill, cntr, barbs]

# Set the attributes for the panel
# and put the panel in the figure
pc = declarative.PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

# Show the figure
pc.show()

### Uncomment line below and SAVE FIGURE - Change figure name ###
#pc.save(f'test.png', dpi=150, bbox_inches='tight')
########################################